# Foundations of Data Science
## Homework 2

Student Name: Harish Puvvada

Student Netid: hp1047
***

### Part 1: Case study
- Read [this article](http://www.nytimes.com/2012/02/19/magazine/shopping-habits.html) in the New York Times.
- Use what we've learned in class and from the book to describe how one could set Target's problem up as a predictive modeling problem, such that they could have gotten the results that they did.  Formulate your solution as a proposed plan using our data science terminology.  Include aspects of the Data Science Workflow that you see as relevant to solving the problem.  Be precise but concise.

Indentifying the Problem: For Target, the predictive modelling problem is to identify the customers or customers with family member in second trimester of pregnancy so that they can draw them to their stores through some special offers.We can predict about this using supervised learning techniques.

Data: Target can collect the required data from customer's online accounts, surveys, advertisements, used coupons,etc. Since this data is collected from different data sources, data has to be transformed, filtered, integrated and cleaned without any duplicates.

Build model: Since we have to identify if a customer is pregnant or not, We can understand that output is discrete(binary).So we can use decision trees, Random forest, k-NN and many other algorithms. We have to divide the available data in to training, validation and testing sets.

Evaluate model: We can train our model using the training data. If our model is giving low accuracy, we might have to analyze more patterns and explore new features available in dataset. We cannot include all the features as we might overfit. For example, We can create a flag basing on all the customers who bought some pregnancy related vitamins and tablets in their first trimester. we have to select the features which can maximize our Information gain. After some iterations and trying different algorithms we will be able to give a solution to Target's problem.

### Part 2: Exploring data in the command line
For this part we will be using the data file located in `"data/advertising_events.csv"`. This file consists of records that pertain to some online advertising events on a given day. There are 4 comma separated columns in this order: `userid`, `timestamp`, `domain`, and `action`. These fields are of type `int` (continuous), `int` (continuous), `string`, and `int` (category) respectively. Answer the following questions using Linux/Unix bash commands. All questions can be answered in one line (sometimes, with pipes)! Some questions will have many possible solutions. Don't forget that in IPython notebooks you must prefix all bash commands with an exclamation point, i.e. `"!command arguments"`.

[Hints: You can experiment with whatever you want in the notebook and then delete things to construct your answer later.  You can also use ssh to use the actual bash shell on EC2 and then just paste your answers here. Recall that once you enter the "!" then filename completion should work.]

1\. How many records (lines) are in this file?

In [3]:
!wc -l advertising_events.csv

   10341 advertising_events.csv


2\. How many unique users are in this file? (hint: consider the 'cut' command and use pipe operator '|')

In [4]:
!cut -d , -f 1 advertising_events.csv | sort | uniq | wc -l

     732


3\. Rank all domains by the number of visits they received in descending order. (hint: consider the 'cut', 'uniq' and 'sort' commands and the pipe operator).

In [5]:
!cut -d , -f 3 advertising_events.csv | sort | uniq -c | sort -r

3114 google.com
2092 facebook.com
1036 youtube.com
1034 yahoo.com
1022 baidu.com
 513 wikipedia.org
 511 amazon.com
 382 qq.com
 321 twitter.com
 316 taobao.com


4\. List all records for the user with user id 37. (hint: this can be done using 'grep')

In [6]:
!grep -w "37" advertising_events.csv

37,648061658,google.com,0
37,642479972,google.com,2
37,644493341,facebook.com,2
37,654941318,facebook.com,1
37,649979874,baidu.com,1
37,653061949,yahoo.com,1
37,655020469,google.com,3
37,640878012,amazon.com,0
37,659864136,youtube.com,1
37,640361378,yahoo.com,1
37,653862134,facebook.com,0
37,648828970,youtube.com,0


### Part 3: Dealing with data Pythonically

In [4]:
# You might find these packages useful. You may import any others you want!
import pandas as pd
import numpy as np

1\. Load the data set `"data/ads_dataset.tsv"` into a Python Pandas data frame called `ads`.

In [5]:
ads = pd.read_csv("ads_dataset.tsv",delim_whitespace=True)

2\. Write a Python function called `getDfSummary()` that does the following:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

Hint: The pandas `describe()` [(manual page)](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) method returns a useful series of values that can be used here.

In [6]:
def getDfSummary(input_data):
    num_nan = input_data.isnull().sum()
    num_distinct = input_data.nunique().transpose()
    data_desc = input_data.describe()
    temp = data_desc.drop('count')
    data = temp.transpose()
    out_data = data.assign(number_distinct= num_distinct)
    output_data = out_data.assign(number_nan=num_nan)
    return output_data

3\. How long does it take for your `getDfSummary()` function to work on your `ads` data frame? Show us the results below.

Hint: `%timeit getDfSummary(ads)`

In [8]:
%timeit getDfSummary(ads)

10 loops, best of 3: 60.8 ms per loop


4\. Using the results returned from `getDfSummary()`, which fields, if any, contain missing `NaN` values?

In [9]:
FinalFrame =getDfSummary(ads)
Fields_with_NaN = []
for key,value in FinalFrame["number_nan"].iteritems():
    if value>0:
        Fields_with_NaN.append(key)
Fields_with_NaN

['buy_freq']

5\. For the fields with missing values, does it look like the data is missing at random? Are there any other fields that correlate perfectly, or predict that the data is missing? What would be an appropriate method for filling in missing values?

Hint: create another data frame that has just the records with a missing value. Get a summary of this data frame using `getDfSummary()` and compare the differences. Do some feature distributions change dramatically?

In [10]:
# NO. It is not looking like data is missing at random. 
# we can see that there is a correlation here between "isbuyer" and "buy_freq". If a customer is not a buyer,then it is
# quite obvious that the customer wont have any buy_frequency.
# one = ads['buy_freq'] >= 1
# ads[one] These two lines of code help us understand the correlation.
# The appropriate method for filling in missing values in "buy_freq" is to insert 0.

new_summary = ads.fillna(0)
#print(getDfSummary(new_summary))
ChangeDF = getDfSummary(new_summary)
ChangeDF.loc["buy_freq"] #this is the only variable changed

# Yes. Feature distributions of "buy_freq" change after filling in the missing values

mean                0.052891
std                 0.298157
min                 0.000000
25%                 0.000000
50%                 0.000000
75%                 0.000000
max                15.000000
number_distinct    11.000000
number_nan          0.000000
Name: buy_freq, dtype: float64

6\. Which variables are binary?

In [11]:
Binary_Variables = []
for key,value in FinalFrame["number_distinct"].iteritems():
    if value ==2 :
        Binary_Variables.append(key)
Binary_Variables

['isbuyer', 'multiple_buy', 'multiple_visit', 'y_buy']